In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import os
import pprint

import torch
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms

import tools._init_paths
from config import cfg
from config import update_config
from core.loss import JointsMSELoss
from core.function import validate
from utils.utils import create_logger

In [2]:
import dataset
import models


In [3]:
import torch.nn.utils.prune as prune

In [4]:
torch.cuda.is_available()

True

In [5]:
def parse_args(args___):
    parser = argparse.ArgumentParser(description='Train keypoints network')
    # general
    parser.add_argument('--cfg',
                        help='experiment configure file name',
                        required=True,
                        type=str)

    parser.add_argument('opts',
                        help="Modify config options using the command-line",
                        default=None,
                        nargs=argparse.REMAINDER)

    parser.add_argument('--modelDir',
                        help='model directory',
                        type=str,
                        default='')
    parser.add_argument('--logDir',
                        help='log directory',
                        type=str,
                        default='')
    parser.add_argument('--dataDir',
                        help='data directory',
                        type=str,
                        default='')
    parser.add_argument('--prevModelDir',
                        help='prev Model directory',
                        type=str,
                        default='')

    args = parser.parse_args(args___)
    return args


In [6]:
config_file_path = 'experiments/coco/transpose_r/TP_R_256x192_d256_h1024_enc4_mh8.yaml'

In [7]:
# config_file_path = 'experiments/coco/transpose_h/TP_H_w32_256x192_stage3_1_4_d64_h128_relu_enc4_mh1.yaml'

In [8]:
args = parse_args(['--cfg', config_file_path, 'TEST.USE_GT_BBOX', 'True'])

In [9]:
update_config(cfg, args)

In [10]:
cfg

CfgNode({'OUTPUT_DIR': 'output', 'LOG_DIR': 'log', 'DATA_DIR': '', 'GPUS': (0,), 'WORKERS': 24, 'PRINT_FREQ': 100, 'AUTO_RESUME': True, 'PIN_MEMORY': True, 'RANK': 0, 'CUDNN': CfgNode({'BENCHMARK': True, 'DETERMINISTIC': False, 'ENABLED': True}), 'MODEL': CfgNode({'NAME': 'transpose_r', 'INIT_WEIGHTS': True, 'PRETRAINED': 'models/pytorch/imagenet/resnet50-19c8e357.pth', 'NUM_JOINTS': 17, 'TAG_PER_JOINT': True, 'TARGET_TYPE': 'gaussian', 'IMAGE_SIZE': [192, 256], 'HEATMAP_SIZE': [48, 64], 'SIGMA': 2, 'EXTRA': CfgNode({'FINAL_CONV_KERNEL': 1, 'DECONV_WITH_BIAS': False, 'NUM_DECONV_LAYERS': 1, 'NUM_DECONV_FILTERS': [256], 'NUM_DECONV_KERNELS': [4], 'NUM_LAYERS': 50}), 'BOTTLENECK_NUM': 0, 'DIM_MODEL': 256, 'DIM_FEEDFORWARD': 1024, 'ENCODER_LAYERS': 4, 'N_HEAD': 8, 'ATTENTION_ACTIVATION': 'relu', 'POS_EMBEDDING': 'sine', 'INTERMEDIATE_SUP': False, 'PE_ONLY_AT_BEGIN': False}), 'LOSS': CfgNode({'USE_OHKM': False, 'TOPK': 8, 'USE_TARGET_WEIGHT': True, 'USE_DIFFERENT_JOINTS_WEIGHT': False}), '

In [11]:
logger, final_output_dir, tb_log_dir = create_logger(cfg, args.cfg, 'valid')

logger.info(pprint.pformat(args))
logger.info(cfg)

Namespace(cfg='experiments/coco/transpose_r/TP_R_256x192_d256_h1024_enc4_mh8.yaml', opts=['TEST.USE_GT_BBOX', 'True'], modelDir='', logDir='', dataDir='', prevModelDir='')
AUTO_RESUME: True
CUDNN:
  BENCHMARK: True
  DETERMINISTIC: False
  ENABLED: True
DATASET:
  COLOR_RGB: True
  DATASET: coco
  DATA_FORMAT: jpg
  FLIP: True
  HYBRID_JOINTS_TYPE: 
  NUM_JOINTS_HALF_BODY: 8
  PROB_HALF_BODY: 0.3
  ROOT: data/coco/
  ROT_FACTOR: 45
  SCALE_FACTOR: 0.35
  SELECT_DATA: False
  TEST_SET: val2017
  TRAIN_SET: train2017
DATA_DIR: 
DEBUG:
  DEBUG: False
  SAVE_BATCH_IMAGES_GT: True
  SAVE_BATCH_IMAGES_PRED: True
  SAVE_HEATMAPS_GT: True
  SAVE_HEATMAPS_PRED: True
GPUS: (0,)
LOG_DIR: log
LOSS:
  TOPK: 8
  USE_DIFFERENT_JOINTS_WEIGHT: False
  USE_OHKM: False
  USE_TARGET_WEIGHT: True
MODEL:
  ATTENTION_ACTIVATION: relu
  BOTTLENECK_NUM: 0
  DIM_FEEDFORWARD: 1024
  DIM_MODEL: 256
  ENCODER_LAYERS: 4
  EXTRA:
    DECONV_WITH_BIAS: False
    FINAL_CONV_KERNEL: 1
    NUM_DECONV_FILTERS: [256]
    

=> creating output/coco/transpose_r/TP_R_256x192_d256_h1024_enc4_mh8
=> creating log/coco/transpose_r/TP_R_256x192_d256_h1024_enc4_mh8_2024-03-22-11-54


In [12]:
cudnn.benchmark = cfg.CUDNN.BENCHMARK
torch.backends.cudnn.deterministic = cfg.CUDNN.DETERMINISTIC
torch.backends.cudnn.enabled = cfg.CUDNN.ENABLED


In [235]:
model = eval('models.'+cfg.MODEL.NAME+'.get_pose_net')(
    cfg, is_train=False
)

==> Add Sine PositionEmbedding~


In [236]:
# model

In [237]:
if cfg.TEST.MODEL_FILE:
    # logger.info('=> loading model from {}'.format(cfg.TEST.MODEL_FILE))
    print(f"loading model weights from cfg.TEST.MODEL_FILE: {cfg.TEST.MODEL_FILE}")
    ckpt_state_dict = torch.load(cfg.TEST.MODEL_FILE)
    # print(ckpt_state_dict['pos_embedding'])  # FOR UNSeen Resolutions
    # ckpt_state_dict.pop('pos_embedding') # FOR UNSeen Resolutions
    model.load_state_dict(ckpt_state_dict, strict=True)   #  strict=False FOR UNSeen Resolutions
else:
    model_state_file = os.path.join(
        final_output_dir, 'final_state.pth'
    )
    print(f"loading model weights from final_output_dir/final_state.pth: {model_state_file}")
    # logger.info('=> loading model from {}'.format(model_state_file))
    model.load_state_dict(torch.load(model_state_file))
w, h = cfg.MODEL.IMAGE_SIZE

loading model weights from cfg.TEST.MODEL_FILE: models/pytorch/transpose_coco/tp_r_256x192_enc4_d256_h1024_mh8.pth


In [238]:
model = torch.nn.DataParallel(model, device_ids=cfg.GPUS).cuda()

# define loss function (criterion) and optimizer
criterion = JointsMSELoss(
    use_target_weight=cfg.LOSS.USE_TARGET_WEIGHT
).cuda()

# Data loading code
normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
)
valid_dataset = eval('dataset.'+cfg.DATASET.DATASET)(
    cfg, cfg.DATASET.ROOT, cfg.DATASET.TEST_SET, False,
    transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ])
)
valid_loader = torch.utils.data.DataLoader(
    valid_dataset,
    batch_size=cfg.TEST.BATCH_SIZE_PER_GPU*len(cfg.GPUS),
    shuffle=False,
    num_workers=cfg.WORKERS,
    pin_memory=True
)

loading annotations into memory...


=> classes: ['__background__', 'person']
=> num_images: 5000


Done (t=0.33s)
creating index...
index created!


=> load 6352 samples


In [239]:
def layer_sparsity(layer):
    count_wts = layer.weight.data.numel()
    count_nz_wts = torch.sum(layer.weight.data.abs().clone().gt(0).float())
    sparsity = round(1.0 - float(count_nz_wts/count_wts), 3)
    print(f"weights total: {count_wts} \t nonzero: {count_nz_wts} \t sparsity: {sparsity}")

In [240]:
def print_net_sparsity_layerwise(dl_net):
    skip_layer_types = [torch.nn.modules.container.Sequential,
                        models.transpose_r.TransPoseR,
                        torch.nn.modules.pooling.AdaptiveAvgPool2d,
                        torch.nn.modules.activation.ReLU,
                        torch.nn.modules.pooling.MaxPool2d,
                        models.transpose_r.Bottleneck,
                        models.transpose_r.TransformerEncoder,
                        torch.nn.modules.container.ModuleList,
                        models.transpose_r.TransformerEncoderLayer,
                        torch.nn.modules.activation.MultiheadAttention,
                        torch.nn.modules.dropout.Dropout
                       ]
    l_i = 0
    for l in dl_net.named_modules():
        skip = False
        l_i += 1
        # print(type(l[1]))
        if l_i > 1 and not 'relu' in l[0].lower() and not 'maxpool' in l[0]:
            for skip_layer in skip_layer_types:
                if isinstance(l[1], skip_layer):
                    skip = True

            if skip:
                continue
            # if  isinstance(l[1], torch.nn.modules.container.Sequential):
            #     continue
            # # if  isinstance(l[1], models.resnet.BasicBlock):
            # #     continue
            # if isinstance(l[1], models.transpose_r.TransPoseR):
            #     continue
            # if  isinstance(l[1], torch.nn.modules.pooling.AdaptiveAvgPool2d):
            #     continue
            # if isinstance(l[1], torch.nn.modules.activation.ReLU):
            #     continue
            # if isinstance(l[1], torch.nn.modules.pooling.MaxPool2d):
            #     continue
                
            print(l_i, end="\t")
            print(l[0], end="\t" )
            # print(type(l[1]), "\t")
            layer_sparsity(l[1])

In [241]:
def prune_model(model, prune_ratios):
    l_i = 0
    for l in model.named_modules():
        l_i += 1
        if l[0] in prune_ratios.keys():
            print(l_i, end="\t")
            print(l[0], end="\t" )
            print(type(l[1]), "\t")
            prune.ln_structured(l[1], 'weight', amount=prune_ratios[l[0]], dim=1, n=float('-inf'))
            print("^^^")

In [242]:
prune_ratios_set1 = {}

prune_ratios_set1['module.layer1.0.conv1'] = 0
prune_ratios_set1['module.layer1.0.conv2'] = 0
prune_ratios_set1['module.layer1.0.conv3'] = 0

prune_ratios_set1['module.layer1.1.conv1'] = 0
prune_ratios_set1['module.layer1.1.conv2'] = 0
prune_ratios_set1['module.layer1.1.conv3'] = 0

prune_ratios_set1['module.layer1.2.conv1'] = 0
prune_ratios_set1['module.layer1.2.conv2'] = 0
prune_ratios_set1['module.layer1.2.conv3'] = 0

prune_ratios_set1['module.layer2.1.conv1'] = 0.5
prune_ratios_set1['module.layer2.1.conv2'] = 0.5
prune_ratios_set1['module.layer2.1.conv3'] = 0.5

prune_ratios_set1['module.layer2.2.conv1'] = 0.5
prune_ratios_set1['module.layer2.2.conv2'] = 0.5
prune_ratios_set1['module.layer2.2.conv3'] = 0.5


In [243]:
prune_model(model, prune_ratios_set1)

9	module.layer1.0.conv1	<class 'torch.nn.modules.conv.Conv2d'> 	
^^^
11	module.layer1.0.conv2	<class 'torch.nn.modules.conv.Conv2d'> 	
^^^
13	module.layer1.0.conv3	<class 'torch.nn.modules.conv.Conv2d'> 	
^^^
20	module.layer1.1.conv1	<class 'torch.nn.modules.conv.Conv2d'> 	
^^^
22	module.layer1.1.conv2	<class 'torch.nn.modules.conv.Conv2d'> 	
^^^
24	module.layer1.1.conv3	<class 'torch.nn.modules.conv.Conv2d'> 	
^^^
28	module.layer1.2.conv1	<class 'torch.nn.modules.conv.Conv2d'> 	
^^^
30	module.layer1.2.conv2	<class 'torch.nn.modules.conv.Conv2d'> 	
^^^
32	module.layer1.2.conv3	<class 'torch.nn.modules.conv.Conv2d'> 	
^^^
48	module.layer2.1.conv1	<class 'torch.nn.modules.conv.Conv2d'> 	
^^^
50	module.layer2.1.conv2	<class 'torch.nn.modules.conv.Conv2d'> 	
^^^
52	module.layer2.1.conv3	<class 'torch.nn.modules.conv.Conv2d'> 	
^^^
56	module.layer2.2.conv1	<class 'torch.nn.modules.conv.Conv2d'> 	
^^^
58	module.layer2.2.conv2	<class 'torch.nn.modules.conv.Conv2d'> 	
^^^
60	module.layer2.2.co

In [244]:
print_net_sparsity_layerwise(model)

3	module.conv1	weights total: 9408 	 nonzero: 9408.0 	 sparsity: 0.0
4	module.bn1	weights total: 64 	 nonzero: 64.0 	 sparsity: 0.0
9	module.layer1.0.conv1	weights total: 4096 	 nonzero: 4096.0 	 sparsity: 0.0
10	module.layer1.0.bn1	weights total: 64 	 nonzero: 64.0 	 sparsity: 0.0
11	module.layer1.0.conv2	weights total: 36864 	 nonzero: 36864.0 	 sparsity: 0.0
12	module.layer1.0.bn2	weights total: 64 	 nonzero: 64.0 	 sparsity: 0.0
13	module.layer1.0.conv3	weights total: 16384 	 nonzero: 16384.0 	 sparsity: 0.0
14	module.layer1.0.bn3	weights total: 256 	 nonzero: 256.0 	 sparsity: 0.0
17	module.layer1.0.downsample.0	weights total: 16384 	 nonzero: 16384.0 	 sparsity: 0.0
18	module.layer1.0.downsample.1	weights total: 256 	 nonzero: 256.0 	 sparsity: 0.0
20	module.layer1.1.conv1	weights total: 16384 	 nonzero: 16384.0 	 sparsity: 0.0
21	module.layer1.1.bn1	weights total: 64 	 nonzero: 64.0 	 sparsity: 0.0
22	module.layer1.1.conv2	weights total: 36864 	 nonzero: 36864.0 	 sparsity: 0.0


In [245]:
validate(cfg, valid_loader, valid_dataset, model, criterion,
             final_output_dir, tb_log_dir)

Test: [0/794]	Time 1.288 (1.288)	Loss 0.0009 (0.0009)	Accuracy 0.607 (0.607)
Test: [100/794]	Time 0.230 (0.242)	Loss 0.0011 (0.0010)	Accuracy 0.484 (0.358)
Test: [200/794]	Time 0.229 (0.235)	Loss 0.0014 (0.0010)	Accuracy 0.227 (0.346)
Test: [300/794]	Time 0.231 (0.233)	Loss 0.0014 (0.0010)	Accuracy 0.268 (0.338)
Test: [400/794]	Time 0.236 (0.233)	Loss 0.0007 (0.0010)	Accuracy 0.584 (0.339)
Test: [500/794]	Time 0.235 (0.233)	Loss 0.0013 (0.0010)	Accuracy 0.438 (0.337)
Test: [600/794]	Time 0.248 (0.234)	Loss 0.0012 (0.0010)	Accuracy 0.480 (0.336)
Test: [700/794]	Time 0.256 (0.235)	Loss 0.0013 (0.0010)	Accuracy 0.339 (0.336)
=> writing results json to output/coco/transpose_r/TP_R_256x192_d256_h1024_enc4_mh8/results/keypoints_val2017_results_0.json


Loading and preparing results...
DONE (t=0.42s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *keypoints*


| Arch | AP | Ap .5 | AP .75 | AP (M) | AP (L) | AR | AR .5 | AR .75 | AR (M) | AR (L) |
|---|---|---|---|---|---|---|---|---|---|---|
| transpose_r | 0.082 | 0.202 | 0.054 | 0.060 | 0.114 | 0.111 | 0.252 | 0.087 | 0.083 | 0.151 |


DONE (t=2.50s).
Accumulating evaluation results...
DONE (t=0.06s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets= 20 ] = 0.082
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets= 20 ] = 0.202
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets= 20 ] = 0.054
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets= 20 ] = 0.060
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets= 20 ] = 0.114
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 20 ] = 0.111
 Average Recall     (AR) @[ IoU=0.50      | area=   all | maxDets= 20 ] = 0.252
 Average Recall     (AR) @[ IoU=0.75      | area=   all | maxDets= 20 ] = 0.087
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets= 20 ] = 0.083
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets= 20 ] = 0.151


0.08215065935656146